In [1]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import pprint
import folium
import seaborn as sns
from collections import defaultdict
from watson_developer_cloud import ToneAnalyzerV3

conn = sqlite3.connect('dataset/database.sqlite')

ModuleNotFoundError: No module named 'watson_developer_cloud'

In [3]:
checkins = pd.read_sql_query("select * from checkin", conn)

In [4]:
businesses = pd.read_sql_query("select * from business", conn)

In [5]:
reviews = pd.read_sql_query("select * from review", conn)

In [6]:
# for i, business in businesses.iterrows():
#     value = len(checkins.loc[checkins['business_id']==business['id']])
#     print (value)
#     businesses.set_value(i, 'checkins', value)

display(checkins[:3])

,business_id,date,count
0,7KPBkxAOEtb3QeIL9PEErg,Thursday-21:00,4
1,7KPBkxAOEtb3QeIL9PEErg,Thursday-1:00,1
2,7KPBkxAOEtb3QeIL9PEErg,Thursday-4:00,1


In [7]:
d = defaultdict(int)

# why do we need i and index
for i, (index, checkin) in enumerate(checkins.iterrows()):
    d[checkin['business_id']] += checkin['count']
    if i % 10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [8]:
l = [(k, v) for (k, v) in d.items()]
f = pd.DataFrame(l)
display(f[:3])

,0,1
0,7KPBkxAOEtb3QeIL9PEErg,113
1,kREVIrSBbtqBhIYkTccQUg,7
2,tJRDll5yqpZwehenzE2cSg,4


In [9]:
id_to_name = dict(zip(businesses['id'], businesses['name']))
id_to_review_count = dict(zip(businesses['id'], businesses['review_count']))
f['name'] = f[0].map(id_to_name)
f['review_count'] = f[0].map(id_to_review_count)
display(f[:5])

,0,1,name,review_count
0,7KPBkxAOEtb3QeIL9PEErg,113,QuikTrip,9
1,kREVIrSBbtqBhIYkTccQUg,7,Richard Petty Driving Experience,3
2,tJRDll5yqpZwehenzE2cSg,4,Ransom Bay Inn B & B,5
3,nhZ1HGWD8lMErdn3FuWuTQ,45,Applebee's,13
4,vDoXZGE7p6xAkKQ0XQPvoA,2,Ahern Catering & Banquet Center,4


In [10]:
f[1].corr(f['review_count'])

0.64486078887319387

In [11]:
reviews[:6]

,id,stars,date,text,useful,funny,cool,business_id,user_id
0,----X0BIDP9tA49U3RvdSQ,4,2014-02-17 00:00:00,"Red, white and bleu salad was super yum and a ...",0,0,0,Ue6-WhXvI-_1xUIuapl0zQ,gVmUR8rqUFdbSeZbsg6z_w
1,---0hl58W-sjVTKi5LghGw,4,2016-07-24 00:00:00,Ate the momos during the momo crawl.. Was the ...,0,0,0,Ae4ABFarGMaI5lk1i98A0w,Y6qylbHq8QJmaCRSlKdIog
2,---3OXpexMp0oAg77xWfYA,5,2012-04-07 00:00:00,Pizza here made my night... Good people and gr...,2,0,0,lKq4Qsz13FDcAVgp49uukQ,SnXZkRN9Yf060pNTk1HMDg
3,---65iIIGzHj96QnOh89EQ,5,2015-09-11 00:00:00,Great brisket sandwich as claimed. Weird that ...,0,0,0,6nKR80xEGHYf2UxAe_Cu_g,VcmSgvslHAhqWoEn16wjjw
4,---7WhU-FtzSUOje87Y4uw,5,2016-01-22 00:00:00,"Interesting food, great atmosphere, and great ...",1,1,1,Z_mJYg3vi8cPZHa1J4BALw,NKF9v-r0jd1p0JVi9h2T1w
5,---94vtJ_5o_nikEs6hUjg,5,2014-09-17 00:00:00,One of my absolute favorite restaurants! I usu...,1,0,0,R1PQEK6qvrZVC9qcWfKvDA,c2MQ_LPuvtiiKFR_-OY9pg


In [12]:
reviews['useful'].corr(reviews['funny'])

0.79930634378351939

In [13]:
reviews['useful'].corr(reviews['cool'])

0.84046368888610179

In [15]:
reviews['funny'].corr(reviews['cool'])

0.84479772294412203

In [22]:
sns.set(style="ticks", color_codes=True)
ax = sns.pairplot(reviews[['useful','cool','funny']])

In [23]:
plt.show()

In [24]:
from watson_developer_cloud import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(
  username='d263d98c-4894-414e-ba3e-5fcfc19e457b',
  password='5UP5V88BgmIk',
  version='2017-09-21'
)

tone = tone_analyzer.tone(reviews['text'][2], content_type='text/plain')

print(tone)

ModuleNotFoundError: No module named 'watson_developer_cloud'